In [1]:
# this is the tab-separated file with the domains
!head -20 data/domains.txt

In [2]:
import pandas as pd

# Load the file with the top domains
df = pd.read_csv("data/domains.txt", sep ="\t", header=0)


In [ ]:
import MySQLdb as mdb
import sys

con = mdb.connect(host = 'ipython.ipeirotis.com', 
                  user = 'root', 
                  passwd = 'dwdstudent2015',
                  charset='utf8', 
                  use_unicode=True);

cursor = con.cursor()

db_name = 'citidomains'
# Create a database
create_db_query = '''CREATE DATABASE IF NOT EXISTS {0} 
                            DEFAULT CHARACTER SET "utf8"
                    '''.format(db_name)
cursor.execute(create_db_query)

cursor.close()


In [ ]:

cursor = con.cursor()
db_name = 'citidomains'
table_name = 'domains'
# Create a table
# The {0} and {1} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {0}.{1} 
                                (
                                domain varchar(255), 
                                count int,
                                json text,
                                organization varchar(255),
                                PRIMARY KEY(domain)
                                )'''.format(db_name, table_name)
cursor.execute(create_table_query)
cursor.close()


In [ ]:

# Inserts the details about the domain in the MySQL database
def insert_into_db(domain, count, json, organization):
    
    query_template = "INSERT INTO citidomains.domains(domain, count, json, organization) VALUES (%s, %s, %s, %s)"
    query_parameters = (domain, count, json, organization)
    
    cursor = con.cursor()
    cursor.execute(query_template, query_parameters)
    con.commit()
    cursor.close()


In [ ]:
# Returns the record for the domain from the MySQL database, if any
def fetch_from_db(domain):
    
    query_template = "SELECT * FROM citidomains.domains WHERE domain='%s'" % (domain)
    cur = con.cursor(mdb.cursors.DictCursor)
    cur.execute(query_template)

    rows = cur.fetchall()
    
    if len(rows)==0:
        return None
    
    for row in rows:
        result = {
            "domain": row["domain"],
            "organization" : row["organization"],
            "count" : row["count"]
        }
        return result
    
print fetch_from_db("gmail.com")



In [4]:

import requests
import json
import collections
import pprint

# Runs the WHOIS query against the web service and returns the JSON object
def whois(domain):
    url = "http://www.whoisxmlapi.com/whoisserver/WhoisService"
    headers = {
      "Accept": "application/json"
    }
    parameters = {
        'domainName': domain,
        'username': 'ipeirotis',
        'password': 'detectica123!',
        'outputFormat': 'JSON'
    }
    resp = requests.get(url, params=parameters, headers=headers)

    data = json.loads(resp.text)
    return data


# Takes the hierarchical JSON object and returns a flat representation
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)



# Processes the flattened JSON file to find the key that is likely to contain
# the organization for the domain
def get_organization(data):
    for key in data:
        if key.count("organization")>0:
            if data[key].count("PRIVACY")==0:
                return data[key]
    for key in data:
        if key.count("registrant_name")>0:
            if data[key].count("PRIVACY")==0:
                return data[key]    
    return None
        
data = whois("nyu.edu")
organization = get_organization(flatten(data))
print(organization)



New York University
ITS Communications Operations Services


In [ ]:
# Go through the list of domains
for index, row in df.iterrows():
    # Print a progress indicator
    if (index%100==0):
        print index
    
    # Do not resolve domains that have less than 100 distinct emails in the dataset
    count =  int(row[0])
    if count<100:
        break
    
    # If the domain is not in the database execute a query and store the results
    # in the database
    domain = row[1]
    if fetch_from_db(domain) is None:
        data = whois(domain)
        organization = get_organization(flatten(data))
        insert_into_db(domain, count, json.dumps(data), organization)

    

